# FDS Challenge: Starter Notebook

This notebook will guide you through the first steps of the competition. Our goal here is to show you how to:

1.  Load the `train.jsonl` and `test.jsonl` files from the competition data.
2.  Create a very simple set of features from the data.
3.  Train a basic model.
4.  Generate a `submission.csv` file in the correct format.
5.  Submit your results.

Let's get started!

In [ ]:
from typing import Any
import json
import os
from pprint import pprint
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_validate

pd.set_option("display.max_columns", 0)

### 1. Loading and Inspecting the Data

When you create a notebook within a Kaggle competition, the competition's data is automatically attached and available in the `../input/` directory.

The dataset is in a `.jsonl` format, which means each line is a separate JSON object. This is great because we can process it one line at a time without needing to load the entire large file into memory.

Let's write a simple loop to load the training data and inspect the first battle.

In [38]:
COMPETITION_NAME = 'fds-pokemon-battles-prediction-2025'
DATA_PATH = os.getcwd() #os.path.join('../input', COMPETITION_NAME)
UNTOUCHED = {'battle_id', 'player_won'}
INFORMATIVE = {
    "p1_unique_pokemon",
    "p2_unique_pokemon",
    "final_p1_hp",  
    "p1_fainted_count", 
    "p1_turns_statused", 
    "p1_missed_turns", 
    "p2_turns_statused",
    "p2_missed_turns", 
    "battle_id",
    "player_won"
}

train_file_path = os.path.join(DATA_PATH, 'train.jsonl')
test_file_path = os.path.join(DATA_PATH, 'test.jsonl')

print(f"Loading data from '{train_file_path}'...")
try:
    with open(train_file_path, 'r', encoding="utf-8") as f:
        train_data = [json.loads(line) for line in f]

    print(f"Successfully loaded {len(train_data)} battles.")

    #print("\n--- Structure of the first train battle: ---")
    if train_data:
        first_battle = train_data[0]
        
        battle_for_display = first_battle.copy()
        battle_for_display['battle_timeline'] = battle_for_display.get('battle_timeline', [])[:2] # Show first 2 turns
        
        #pprint(battle_for_display)
        if len(first_battle.get('battle_timeline', [])) > 3:
            print("    ...")
            print("    (battle_timeline has been truncated for display)")

except FileNotFoundError:
    print(f"ERROR: Could not find the training file at '{train_file_path}'.")
    print("Please make sure you have added the competition data to this notebook.")

Loading data from 'c:\Users\stefa\PycharmProjects\pokemon-challenge\train.jsonl'...
Successfully loaded 10000 battles.
    ...
    (battle_timeline has been truncated for display)


### 2. Basic Feature Engineering

A successful model will likely require creating many complex features. For this starter notebook, however, we will create a very simple feature set based **only on the initial team stats**. This will be enough to train a model and generate a submission file.

It's up to you to engineer more powerful features!

In [39]:
def features_check(data: dict) -> None:
    print("All battles have at least one turn: ", all(all(turn for turn in battle.get('battle_timeline', False)) for battle in data))
    print("All battles' turns have at least one P1 move: ", 
        all((
            any((turn.get("p1_move_details", False) for turn in battle.get('battle_timeline', False))) for battle in data
        ))
    )
    print("All battles' turns have at least one P2 move: ", 
        all((
            any((turn.get("p2_move_details", False) for turn in battle.get('battle_timeline', False))) for battle in data
        ))
    )
    print("player_won feature always exists: ", all(('player_won' in battle for battle in data)))
    print("P1 Team always exists: ", all(battle.get('p1_team_details', False) for battle in data))
    print("P2 Team always exists: ", all(battle.get('p2_team_details', False) for battle in data))
    
    return None

In [40]:
def agg_pokemons_stats(prefix: str, stats: dict[str, Any]):
    return {
        f"{prefix}_mean_power": np.mean(stats["powers"]) if stats["powers"] else 0,
        f"{prefix}_mean_accuracy": np.mean(stats["accuracy"]) if stats["accuracy"] else 0,
        f"{prefix}_lost_hp": stats["lost_hp"],
        f"{prefix}_turns_statused": stats["turns_statused"],
        f"{prefix}_missed_turns": stats["missed_turns"],
        f"{prefix}_switches": stats["switches"],
        f"{prefix}_net_boost": stats["net_boost"],
    }

In [41]:
def create_features(data: list[dict]) -> pd.DataFrame:
    feature_list = []
    
    features_check(data)

    for battle in data:
        features = {}

        p1_stats = {
            "powers": [], "accuracy": [], "hp_t0": {}, "lost_hp": 0, "turns_statused": 0,
            "missed_turns": 0, "priority": 0, "switches": 0, "net_boost": 0,
            "base_boosts": {"atk": 0, "def": 0, "spa": 0, "spd": 0, "spe": 0}
        }
        
        p2_stats = {
            "powers": [], "accuracy": [], "hp_t0": {}, "lost_hp": 0, "turns_statused": 0,
            "missed_turns": 0, "priority": 0, "switches": 0, "net_boost": 0,
            "base_boosts": {"atk": 0, "def": 0, "spa": 0, "spd": 0, "spe": 0}
        }

        # --- Initial Pokémon states ---
        first_turn = battle["battle_timeline"][0]
        p1_lead = first_turn.get("p1_pokemon_state", {}).get("name", "")
        p2_lead = battle.get("p2_lead_details", {}).get("name", "")
        p1_prev_hp = first_turn.get("p1_pokemon_state", {}).get("hp_pct", 1.0)
        p2_prev_hp = first_turn.get("p2_pokemon_state", {}).get("hp_pct", 1.0)
        
        # --- Player 1 Team Features ---
        p1_team = battle.get('p1_team_details', None)
        features['p1_mean_hp'] = np.mean([p.get('base_hp') for p in p1_team])
        features['p1_mean_spe'] = np.mean([p.get('base_spe') for p in p1_team])
        features['p1_mean_atk'] = np.mean([p.get('base_atk') for p in p1_team])
        features['p1_mean_def'] = np.mean([p.get('base_def') for p in p1_team])

        # --- Player 2 Lead Features ---
        if p2_lead := battle.get('p2_lead_details'):
            # Player 2's lead Pokémon's stats
            features['p2_lead_hp'] = p2_lead.get('base_hp')
            features['p2_lead_spe'] = p2_lead.get('base_spe')
            features['p2_lead_atk'] = p2_lead.get('base_atk')
            features['p2_lead_def'] = p2_lead.get('base_def')
        
        # --- Battle Timeline Features ---
        if timeline := battle.get('battle_timeline', []):
            p1_names = [t['p1_pokemon_state']['name'] for t in timeline if t.get('p1_pokemon_state')]
            p1_moves = [t['p1_move_details']['name'] for t in timeline if t.get('p1_move_details')]
            p2_names = [t['p2_pokemon_state']['name'] for t in timeline if t.get('p2_pokemon_state')]

            # Unique Pokémons
            features['p1_unique_pokemon'] = len(set(p1_names))
            #features['p1_unique_moves'] = len(set(p1_moves))
            features['p2_unique_pokemon'] = len(set(p2_names))

            # Compute damage dealt (approximate)
            # delta HP of opponent between turns
            p2_hp_deltas = []
            for t, t_stats in enumerate(timeline):
                p1_state = t_stats.get("p1_pokemon_state", {})
                p2_state = t_stats.get("p2_pokemon_state", {})

                # --- Moves and accuracy ---
                for player, stats, move_key in [
                    ("p1", p1_stats, "p1_move_details"),
                    ("p2", p2_stats, "p2_move_details")
                ]:
                    move = t_stats.get(move_key)
                    if move:
                        stats["powers"].append(move.get("base_power", 0))
                        stats["accuracy"].append(move.get("accuracy", 0))
                    else:
                        stats["missed_turns"] += 1

                # --- Status tracking ---
                if p1_state.get("status") != "nostatus":
                    p1_stats["turns_statused"] += 1
                if p2_state.get("status") != "nostatus":
                    p2_stats["turns_statused"] += 1
                
                # --- HP and damage tracking ---
                p1_name, p2_name = p1_state.get("name", ""), p2_state.get("name", "")
                p1_hp, p2_hp = p1_state.get("hp_pct", 1.0), p2_state.get("hp_pct", 1.0)

                # --- Switches ---
                if p1_name != p1_lead and p1_prev_hp > 0:
                    p1_stats["switches"] += 1

                if p2_name != p2_lead and p2_prev_hp > 0:
                    p2_stats["switches"] += 1

                # --- Boost tracking ---
                if p1_name != p1_lead:
                    p1_stats["base_boosts"] = {k: 0 for k in p1_stats["base_boosts"]}
                if p2_name != p2_lead:
                    p2_stats["base_boosts"] = {k: 0 for k in p2_stats["base_boosts"]}

                p1_boosts = p1_state.get("boosts", {})
                p2_boosts = p2_state.get("boosts", {})

                for stat in ["atk", "def", "spa", "spd", "spe"]:
                    p1_stats["net_boost"] += (p1_boosts.get(stat, 0) - p1_stats["base_boosts"].get(stat, 0))
                    p2_stats["net_boost"] += (p2_boosts.get(stat, 0) - p2_stats["base_boosts"].get(stat, 0))
                
                prev_hp = timeline[t-1]['p2_pokemon_state']['hp_pct']
                curr_hp = timeline[t]['p2_pokemon_state']['hp_pct']
                p2_hp_deltas.append(prev_hp - curr_hp)
            features['mean_damage_dealt'] = np.mean([d for d in p2_hp_deltas if d > 0]) if p2_hp_deltas else None

            # Final HP and KO counts
            last_state = timeline[-1]['p1_pokemon_state']
            features['final_p1_hp'] = last_state.get('hp_pct', None)
            features['p1_fainted_count'] = sum(t['p1_pokemon_state']['status'] == 'fnt' for t in timeline)
            features['p2_fainted_count'] = sum(t['p2_pokemon_state']['status'] == 'fnt' for t in timeline)
            
        else:
            features.update({
                'p1_unique_pokemon': None,
                'p1_unique_moves': None,
                'p2_unique_pokemon': None,
                'mean_damage_dealt': None,
                'final_p1_hp': None,
                'p1_fainted_count': None,
                'p2_fainted_count': None,
            })

        features.update(agg_pokemons_stats("p1", p1_stats))
        features.update(agg_pokemons_stats("p2", p2_stats))


        features['battle_id'] = battle.get('battle_id')
        if 'player_won' in battle:
            features['player_won'] = int(battle['player_won'])
            
        feature_list.append(features)
        
    return pd.DataFrame(feature_list) #.fillna(0) #NOTE accuracy improvement of 0.01

In [42]:
print("Processing training data...")
train_df = create_features(train_data)

print("\nProcessing test data...")
with open(test_file_path, 'r', encoding="utf-8") as f:
    test_data = [json.loads(line) for line in f]
        
test_df = create_features(test_data)

Processing training data...
All battles have at least one turn:  True
All battles' turns have at least one P1 move:  False
All battles' turns have at least one P2 move:  False
player_won feature always exists:  True
P1 Team always exists:  True
P2 Team always exists:  False


c:\Users\stefa\anaconda3\envs\sapienza\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\stefa\anaconda3\envs\sapienza\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)



Processing test data...
All battles have at least one turn:  True
All battles' turns have at least one P1 move:  False
All battles' turns have at least one P2 move:  False
player_won feature always exists:  False
P1 Team always exists:  True
P2 Team always exists:  False


In [43]:

#train_df = train_df.drop(columns=)
#test_df = test_df.drop(columns=)
#train_df.corr()

In [44]:
keepers = list(set(train_df.columns).difference(UNTOUCHED))

In [45]:
scaler = StandardScaler(with_mean=True, with_std=True)

train_df[keepers] = scaler.fit_transform(train_df[keepers])
test_df[keepers] = scaler.fit_transform(test_df[keepers])

In [46]:
print("\nTraining dataset preview:")
display(train_df.head())
display(train_df.describe())
display(train_df.dtypes)

print("\nTesting dataset preview:")
display(test_df.head())
display(test_df.describe())
display(test_df.dtypes)


Training dataset preview:


,p1_mean_hp,p1_mean_spe,p1_mean_atk,p1_mean_def,p2_lead_hp,p2_lead_spe,p2_lead_atk,p2_lead_def,p1_unique_pokemon,p2_unique_pokemon,mean_damage_dealt,final_p1_hp,p1_fainted_count,p2_fainted_count,p1_mean_power,p1_mean_accuracy,p1_lost_hp,p1_turns_statused,p1_missed_turns,p1_switches,p1_net_boost,p2_mean_power,p2_mean_accuracy,p2_lost_hp,p2_turns_statused,p2_missed_turns,p2_switches,p2_net_boost,battle_id,player_won
0,0.202093,0.520813,-0.732064,-0.745443,-0.279683,0.490498,0.642099,1.185993,-1.404495,-1.331840,-0.308829,-0.821146,-0.956146,1.113808,-0.118412,-0.823055,0.0,-1.089872,-1.109216,-0.142418,-0.135324,0.446467,0.785598,0.0,1.042992,2.443488,0.093138,-0.318420,0,1
1,0.761596,-1.738011,-0.732064,-0.492591,-0.450188,0.723525,-0.711192,-0.611967,0.918520,0.950271,-1.393217,-0.382753,0.405885,-0.600270,1.450473,0.172684,0.0,-0.341616,0.192108,0.039076,-0.135324,0.170319,0.322559,0.0,-1.090316,0.158909,0.093138,-0.318420,1,1
2,0.823762,-1.224642,0.906915,0.097399,6.199509,-2.538851,-3.147117,-2.409928,-2.566002,-1.331840,-0.661738,-0.189724,-0.956146,-0.600270,-1.285978,-0.326265,0.0,0.406640,-1.109216,-1.594365,0.308258,-0.309510,-0.358604,0.0,0.509665,0.485277,0.093138,-0.395747,2,1
3,0.637262,0.007444,-0.029644,-0.492591,0.231832,0.257471,1.995390,1.635483,-0.242987,-1.331840,0.158946,-1.513355,0.405885,-0.600270,0.404353,-0.060591,0.0,1.341960,0.192108,0.765049,-0.135324,1.549122,-0.440752,0.0,-1.090316,-0.493828,0.093138,-0.163767,3,1
4,0.077760,-0.403251,-0.263784,0.855957,-0.279683,0.490498,0.642099,1.185993,-0.242987,-0.190785,0.321136,1.133907,-0.956146,-0.600270,-1.299584,0.906155,0.0,-0.715744,-0.783885,-1.049885,-0.312756,-0.686812,0.363574,0.0,1.931870,-0.820197,0.093138,-0.163767,4,1


,p1_mean_hp,p1_mean_spe,p1_mean_atk,p1_mean_def,p2_lead_hp,p2_lead_spe,p2_lead_atk,p2_lead_def,p1_unique_pokemon,p2_unique_pokemon,mean_damage_dealt,final_p1_hp,p1_fainted_count,p2_fainted_count,p1_mean_power,p1_mean_accuracy,p1_lost_hp,p1_turns_statused,p1_missed_turns,p1_switches,p1_net_boost,p2_mean_power,p2_mean_accuracy,p2_lost_hp,p2_turns_statused,p2_missed_turns,p2_switches,p2_net_boost,battle_id,player_won
count,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,9.988000e+03,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,10000.0,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,10000.0,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,10000.00000,10000.000000
mean,-4.185097e-16,-9.471535e-16,7.389644e-17,3.140599e-16,5.115908e-17,1.605827e-16,1.012523e-16,7.105427e-19,-1.136868e-16,-2.842171e-16,-3.083903e-16,-2.046363e-16,1.136868e-17,-1.527667e-17,-1.548983e-16,1.150369e-15,0.0,1.136868e-17,-1.364242e-16,-1.747935e-16,1.492140e-17,1.620037e-16,-1.056577e-15,0.0,-9.094947e-17,1.364242e-16,-3.069545e-16,7.105427e-18,4999.50000,0.500000
std,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,0.0,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,0.0,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,2886.89568,0.500025
min,-3.714422e+00,-3.586140e+00,-3.073463e+00,-2.262559e+00,-6.206930e-01,-3.470958e+00,-3.147117e+00,-2.409928e+00,-4.889016e+00,-4.755008e+00,-3.123789e+00,-1.623657e+00,-1.637162e+00,-6.002704e-01,-3.124315e+00,-2.566253e+01,0.0,-2.399320e+00,-2.085208e+00,-4.135274e+00,-2.708099e+00,-3.154841e+00,-2.470957e+01,0.0,-1.979194e+00,-2.125670e+00,-1.073681e+01,-2.483573e+00,0.00000,0.000000
25%,-2.952418e-01,-8.139467e-01,-7.320642e-01,-7.454431e-01,-4.501879e-01,-4.416091e-01,-7.111924e-01,-6.119674e-01,-2.429873e-01,-1.907845e-01,-6.751185e-01,-8.488874e-01,-9.561460e-01,-6.002704e-01,-6.608279e-01,-4.256231e-01,0.0,-7.157443e-01,-7.838847e-01,-3.239114e-01,-2.240399e-01,-6.610847e-01,-4.070766e-01,0.0,-7.347645e-01,-8.201966e-01,9.313754e-02,-2.410936e-01,2499.75000,0.000000
50%,2.642603e-01,7.443852e-03,-2.637844e-01,-1.554538e-01,-2.796828e-01,4.904982e-01,1.007823e-01,6.226787e-02,-2.429873e-01,-1.907845e-01,-7.287296e-02,1.963350e-01,-2.751303e-01,-6.002704e-01,4.948732e-02,2.139934e-01,0.0,3.251172e-02,-1.332230e-01,2.205690e-01,-1.353235e-01,6.870744e-02,1.939377e-01,0.0,-2.366194e-02,-1.674597e-01,9.313754e-02,-1.637667e-01,4999.50000,0.500000
75%,6.372618e-01,5.208129e-01,5.557053e-01,5.188198e-01,-1.091778e-01,7.235250e-01,6.420988e-01,1.185993e+00,9.185200e-01,9.502713e-01,5.653945e-01,1.133907e+00,4.058853e-01,1.113808e+00,7.220467e-01,6.812239e-01,0.0,7.807677e-01,5.174387e-01,5.835559e-01,-1.353235e-01,7.192934e-01,6.389051e-01,0.0,6.874406e-01,4.852773e-01,9.313754e-02,-1.637667e-01,7499.25000,1.000000
max,1.694099e+00,4.319744e+00,4.629740e+00,4.227324e+00,6.199509e+00,1.189579e+00,3.835866e+00,5.456150e+00,9.185200e-01,9.502713e-01,7.259643e+00,1.133907e+00,2.448932e+00,6.256045e+00,3.901601e+00,1.164103e+00,0.0,3.212600e+00,7.674717e+00,1.128036e+00,1.423673e+01,4.335880e+00,1.108321e+00,0.0,2.998524e+00,7.665383e+00,9.313754e-02,1.313646e+01,9999.00000,1.000000


p1_mean_hp           float64
p1_mean_spe          float64
p1_mean_atk          float64
p1_mean_def          float64
p2_lead_hp           float64
p2_lead_spe          float64
p2_lead_atk          float64
p2_lead_def          float64
p1_unique_pokemon    float64
p2_unique_pokemon    float64
mean_damage_dealt    float64
final_p1_hp          float64
p1_fainted_count     float64
p2_fainted_count     float64
p1_mean_power        float64
p1_mean_accuracy     float64
p1_lost_hp           float64
p1_turns_statused    float64
p1_missed_turns      float64
p1_switches          float64
p1_net_boost         float64
p2_mean_power        float64
p2_mean_accuracy     float64
p2_lost_hp           float64
p2_turns_statused    float64
p2_missed_turns      float64
p2_switches          float64
p2_net_boost         float64
battle_id              int64
player_won             int64
dtype: object


Testing dataset preview:


,p1_mean_hp,p1_mean_spe,p1_mean_atk,p1_mean_def,p2_lead_hp,p2_lead_spe,p2_lead_atk,p2_lead_def,p1_unique_pokemon,p2_unique_pokemon,mean_damage_dealt,final_p1_hp,p1_fainted_count,p2_fainted_count,p1_mean_power,p1_mean_accuracy,p1_lost_hp,p1_turns_statused,p1_missed_turns,p1_switches,p1_net_boost,p2_mean_power,p2_mean_accuracy,p2_lost_hp,p2_turns_statused,p2_missed_turns,p2_switches,p2_net_boost,battle_id
0,0.347635,0.297212,-0.497947,-0.937347,-0.111797,1.206605,0.120196,0.067841,-0.252920,-0.210231,0.358433,1.132883,1.082543,-0.602802,-0.069245,0.803228,0.0,1.155070,1.166160,0.216933,-0.147568,0.672315,-0.350403,0.0,1.010900,-0.172866,0.097413,1.526410,0
1,-3.135315,2.448247,2.499830,2.635469,-0.437793,0.730007,-0.697689,-0.607419,-1.414171,0.926151,-1.763069,1.132883,-0.957938,-0.602802,-0.312162,-2.037690,0.0,-1.600432,-1.116337,0.952675,3.230791,1.285556,1.153320,0.0,0.313102,3.749966,0.097413,-0.147762,1
2,0.531594,-1.751393,1.825911,1.784799,-0.437793,0.730007,-0.697689,-0.607419,-0.252920,0.926151,-2.366457,1.132883,-0.957938,-0.602802,-1.296566,-3.555448,0.0,-2.151533,-0.464195,0.400869,-0.147568,-0.480580,-0.290254,0.0,1.010900,-0.499769,0.097413,0.354489,2
3,0.102357,-0.522230,-1.078912,0.083458,2.985166,-3.559383,2.573852,0.292928,-2.575421,-0.210231,0.457273,-0.736347,-1.638098,-0.602802,-1.357988,0.803228,0.0,1.155070,-1.442408,-0.150937,-0.312366,0.683265,0.707772,0.0,1.010900,-0.172866,0.097413,-0.315180,3
4,0.286316,0.297212,-0.381754,-0.512012,-0.274795,0.253408,0.120196,0.067841,-0.252920,0.926151,1.211936,-1.094245,0.402383,1.163909,0.755483,0.528891,0.0,-0.314531,-0.138124,0.032998,-0.229967,0.208003,-0.684564,0.0,-0.384696,-0.172866,0.097413,-0.231471,4


,p1_mean_hp,p1_mean_spe,p1_mean_atk,p1_mean_def,p2_lead_hp,p2_lead_spe,p2_lead_atk,p2_lead_def,p1_unique_pokemon,p2_unique_pokemon,mean_damage_dealt,final_p1_hp,p1_fainted_count,p2_fainted_count,p1_mean_power,p1_mean_accuracy,p1_lost_hp,p1_turns_statused,p1_missed_turns,p1_switches,p1_net_boost,p2_mean_power,p2_mean_accuracy,p2_lost_hp,p2_turns_statused,p2_missed_turns,p2_switches,p2_net_boost,battle_id
count,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,4.998000e+03,5.000000e+03,5000.000000,5.000000e+03,5.000000e+03,5.000000e+03,5000.0,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5000.0,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5000.000000
mean,-9.123369e-16,1.355005e-15,3.637979e-16,-7.560175e-16,-4.831691e-17,-3.467449e-16,-2.273737e-17,-9.166001e-17,-5.002221e-16,4.774847e-16,-1.734418e-16,1.136868e-17,0.000000,2.557954e-17,3.836931e-17,1.720935e-15,0.0,1.364242e-16,-1.818989e-16,1.847411e-16,5.684342e-18,3.581135e-16,-1.016076e-15,0.0,4.547474e-17,-1.136868e-16,-3.126388e-17,-5.684342e-18,2499.500000
std,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100,1.000100e+00,1.000100e+00,1.000100e+00,0.0,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,0.0,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1443.520003
min,-3.638136e+00,-3.595138e+00,-2.821806e+00,-2.213353e+00,-6.007913e-01,-3.559383e+00,-3.151344e+00,-2.408115e+00,-4.897923e+00,-4.755759e+00,-2.829711e+00,-1.615985e+00,-1.638098,-6.028019e-01,-3.120609e+00,-2.520398e+01,0.0,-2.335233e+00,-2.094550e+00,-4.197516e+00,-2.207544e+00,-3.182349e+00,-2.557953e+01,0.0,-1.954741e+00,-2.134282e+00,-1.026562e+01,-5.002863e+00,0.000000
25%,-3.268798e-01,-8.295211e-01,-7.303329e-01,-7.672129e-01,-4.377932e-01,-4.614907e-01,-6.976887e-01,-6.074195e-01,-2.529204e-01,-2.102307e-01,-6.672999e-01,-8.463018e-01,-0.957938,-6.028019e-01,-6.716449e-01,-4.174590e-01,0.0,-6.819316e-01,-7.902658e-01,-3.348728e-01,-2.299674e-01,-6.592964e-01,-4.263489e-01,0.0,-7.335948e-01,-4.997688e-01,9.741252e-02,-2.314711e-01,1249.750000
50%,2.863157e-01,9.235111e-02,-2.655612e-01,-1.717436e-01,-2.747952e-01,4.917071e-01,1.201964e-01,6.784119e-02,-2.529204e-01,-2.102307e-01,-9.486330e-02,1.982678e-01,-0.277778,-6.028019e-01,4.031045e-02,2.106589e-01,0.0,5.286890e-02,-1.381237e-01,2.169334e-01,-1.475684e-01,8.097413e-02,1.909369e-01,0.0,-3.579702e-02,-1.728661e-01,9.741252e-02,-1.477625e-01,2499.500000
75%,6.542330e-01,5.020721e-01,5.477892e-01,5.087928e-01,-1.117971e-01,7.300065e-01,6.654531e-01,1.193276e+00,9.083302e-01,9.261513e-01,5.626491e-01,1.132883e+00,0.402383,1.163909e+00,7.201829e-01,6.744088e-01,0.0,7.876694e-01,5.140184e-01,5.848042e-01,-1.475684e-01,7.336387e-01,6.672677e-01,0.0,6.620007e-01,4.809392e-01,9.741252e-02,-1.477625e-01,3749.250000
max,1.696665e+00,5.009003e+00,3.894145e+00,4.251743e+00,5.919131e+00,1.206605e+00,3.882468e+00,5.469927e+00,9.083302e-01,9.261513e-01,6.024642e+00,1.132883e+00,2.442864,6.464043e+00,3.567549e+00,1.132433e+00,0.0,2.808371e+00,7.687581e+00,1.136610e+00,1.402506e+01,3.105814e+00,1.153320e+00,0.0,3.104293e+00,7.672798e+00,9.741252e-02,1.425012e+01,4999.000000


p1_mean_hp           float64
p1_mean_spe          float64
p1_mean_atk          float64
p1_mean_def          float64
p2_lead_hp           float64
p2_lead_spe          float64
p2_lead_atk          float64
p2_lead_def          float64
p1_unique_pokemon    float64
p2_unique_pokemon    float64
mean_damage_dealt    float64
final_p1_hp          float64
p1_fainted_count     float64
p2_fainted_count     float64
p1_mean_power        float64
p1_mean_accuracy     float64
p1_lost_hp           float64
p1_turns_statused    float64
p1_missed_turns      float64
p1_switches          float64
p1_net_boost         float64
p2_mean_power        float64
p2_mean_accuracy     float64
p2_lost_hp           float64
p2_turns_statused    float64
p2_missed_turns      float64
p2_switches          float64
p2_net_boost         float64
battle_id              int64
dtype: object

### 3. Training Models

In [ ]:
# Define predictor features (X) and target (y)
X_train = train_df[keepers]
print(train_df.columns)
y_train = train_df['player_won']

X_test = test_df[keepers]

print("Training...")
model = XGBClassifier(
    random_state=100,
    n_estimators=200,
    learning_rate=0.05,
    max_depth=3,
    eval_metric='logloss',
    n_jobs=-1
)
model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0)
model.fit(X_train, y_train)
print("Model training complete.")

Index(['p1_mean_hp', 'p1_mean_spe', 'p1_mean_atk', 'p1_mean_def', 'p2_lead_hp',
       'p2_lead_spe', 'p2_lead_atk', 'p2_lead_def', 'p1_unique_pokemon',
       'p2_unique_pokemon', 'mean_damage_dealt', 'final_p1_hp',
       'p1_fainted_count', 'p2_fainted_count', 'p1_mean_power',
       'p1_mean_accuracy', 'p1_lost_hp', 'p1_turns_statused',
       'p1_missed_turns', 'p1_switches', 'p1_net_boost', 'p2_mean_power',
       'p2_mean_accuracy', 'p2_lost_hp', 'p2_turns_statused',
       'p2_missed_turns', 'p2_switches', 'p2_net_boost', 'battle_id',
       'player_won'],
      dtype='object')
Training...
Model training complete.


In [49]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=100)
cv_results = cross_validate(
    model,
    X_train,
    y_train,
    cv=cv,
    scoring={
        "accuracy_score": make_scorer(accuracy_score),
        "precision_score": make_scorer(precision_score),
        "recall_score": make_scorer(recall_score),
        "f1_score": make_scorer(f1_score),
        "roc_auc_score": make_scorer(roc_auc_score)
    },
    return_train_score=True,
    n_jobs=-1
)

results_df = pd.DataFrame(cv_results)
display(results_df)

results_df = pd.DataFrame(cv_results)
summary = results_df.filter(regex='(train_|test_)').describe().loc[['mean', 'std']].T
summary.rename(columns={'mean': 'Mean', 'std': 'Std'}, inplace=True)
display(summary)

,fit_time,score_time,test_accuracy_score,train_accuracy_score,test_precision_score,train_precision_score,test_recall_score,train_recall_score,test_f1_score,train_f1_score,test_roc_auc_score,train_roc_auc_score
0,1.941696,0.022309,0.804,0.851667,0.804000,0.852215,0.804,0.850889,0.804000,0.851551,0.804,0.851667
1,1.907760,0.030193,0.816,0.851889,0.825103,0.851031,0.802,0.853111,0.813387,0.852070,0.816,0.851889
2,2.338782,0.022896,0.822,0.849778,0.819444,0.848384,0.826,0.851778,0.822709,0.850078,0.822,0.849778
3,2.352577,0.023510,0.807,0.850333,0.803960,0.848398,0.812,0.853111,0.807960,0.850748,0.807,0.850333
4,2.102520,0.024005,0.819,0.849778,0.812133,0.848848,0.830,0.851111,0.820969,0.849978,0.819,0.849778
5,1.900064,0.022218,0.830,0.849000,0.838115,0.848767,0.818,0.849333,0.827935,0.849050,0.830,0.849000
6,2.141538,0.026516,0.820,0.849778,0.827869,0.847768,0.808,0.852667,0.817814,0.850211,0.820,0.849778
7,2.268579,0.023580,0.835,0.848778,0.839757,0.848236,0.828,0.849556,0.833837,0.848895,0.835,0.848778
8,0.812482,0.014745,0.831,0.848778,0.821359,0.849321,0.846,0.848000,0.833498,0.848660,0.831,0.848778
9,0.737574,0.020600,0.834,0.848778,0.827451,0.849166,0.844,0.848222,0.835644,0.848694,0.834,0.848778


,Mean,Std
test_accuracy_score,0.821800,0.010830
train_accuracy_score,0.849856,0.001149
test_precision_score,0.821919,0.012483
train_precision_score,0.849213,0.001376
test_recall_score,0.821800,0.015676
train_recall_score,0.850778,0.001927
test_f1_score,0.821775,0.011093
train_f1_score,0.849993,0.001198
test_roc_auc_score,0.821800,0.010830
train_roc_auc_score,0.849856,0.001149


### 4. Creating the Submission File

The competition requires a `.csv` file with two columns: `battle_id` and `player_won`. Let's use our trained model to make predictions on the test set and format them correctly.

In [ ]:
print("Generating predictions on the test set...")
submission_df = pd.DataFrame({
    'battle_id': test_df['battle_id'],
    'player_won': model.predict(X_test)
})

submission_df.to_csv('submission.csv', index=False)

print("\n'submission.csv' file created successfully!")
display(submission_df.head())

Generating predictions on the test set...

'submission.csv' file created successfully!


,battle_id,player_won
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1


### 5. Submitting Your Results

Once you have generated your `submission.csv` file, there are two primary ways to submit it to the competition.

---

#### Method A: Submitting Directly from the Notebook

This is the standard method for code competitions. It ensures that your submission is linked to the code that produced it, which is crucial for reproducibility.

1.  **Save Your Work:** Click the **"Save Version"** button in the top-right corner of the notebook editor.
2.  **Run the Notebook:** In the pop-up window, select **"Save & Run All (Commit)"** and then click the **"Save"** button. This will run your entire notebook from top to bottom and save the output, including your `submission.csv` file.
3.  **Go to the Viewer:** Once the save process is complete, navigate to the notebook viewer page. 
4.  **Submit to Competition:** In the viewer, find the **"Submit to Competition"** section. This is usually located in the header of the output section or in the vertical "..." menu on the right side of the page. Clicking the **Submit** button this will submit your generated `submission.csv` file.

After submitting, you will see your score in the **"Submit to Competition"** section or in the [Public Leaderboard](https://www.kaggle.com/competitions/fds-pokemon-battles-prediction-2025/leaderboard?).

---

#### Method B: Manual Upload

You can also generate your predictions and submission file using any environment you prefer (this notebook, Google Colab, or your local machine).

1.  **Generate the `submission.csv` file** using your model.
2.  **Download the file** to your computer.
3.  **Navigate to the [Leaderboard Page](https://www.kaggle.com/competitions/fds-pokemon-battles-prediction-2025/leaderboard?)** and click on the **"Submit Predictions"** button.
4.  **Upload Your File:** Drag and drop or select your `submission.csv` file to upload it.

This method is quick, but keep in mind that for the final evaluation, you might be required to provide the code that generated your submission.

Good luck!